In [1]:
from ragbuilder.generation.generate_system_prompt import Generator, RAGASEvaluator
from ragbuilder.generation.utils import dummyRetriever
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
llm = AzureChatOpenAI(model="gpt-4o-mini")
# eval_dataset_path = "/Users/ashwinaravind/Desktop/kruxgitrepo/ragbuilder/rag_test_data_lilianweng_gpt-4o_1721032414.736622_SEMI.csv"
eval_dataset_path = "/Users/ashwinaravind/Desktop/kruxgitrepo/ragbuilder/gensimtest.csv"
ragas_evaluator=RAGASEvaluator()


RAGASEvaluator initiated


In [3]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import chromadb

from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain.retrievers import EnsembleRetriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from ragbuilder.generation.evaluation import RAGASEvaluator
def sample_retriever():
    print("rag_get_retriever initiated")
    try:
        def format_docs(docs):
            return "\n".join(doc.page_content for doc in docs)

        # LLM setup
        llm = AzureChatOpenAI(model="gpt-4o-mini")

        # Document loader
        loader = WebBaseLoader("https://raw.githubusercontent.com/ashwinaravind/ashwinaravind.github.io/refs/heads/main/thevanishingtown")
        docs = loader.load()

        # Embedding model
        embedding = AzureOpenAIEmbeddings(model="text-embedding-3-large")

        # Text splitting and embedding storage
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
        splits = splitter.split_documents(docs)

        # Initialize Chroma database
        c = Chroma.from_documents(
            documents=splits,
            embedding=embedding,
            collection_name="testindex-ragbuilder-retreiver",
            client_settings=chromadb.config.Settings(allow_reset=True),
        )

        # Retriever setup
        retriever = c.as_retriever(search_type="similarity", search_kwargs={"k": 5})
        ensemble_retriever = EnsembleRetriever(retrievers=[retriever])
        print("rag_get_retriever completed")
        return ensemble_retriever
    except Exception as e:
        import traceback
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return None

In [4]:
gen=PromptGenerator(
        eval_dataset_path=eval_dataset_path,
        retriever=sample_retriever,
        evaluator=ragas_evaluator,
        prompt_template_path=None,
        read_local=False,
        query_rewrite=True
        llm=llm)
#Find the optimal prompt
best_prompt,max_answer_correctness=gen.optimize()

Fetching prompts from online file: https://raw.githubusercontent.com/ashwinaravind/rag_prompts/refs/heads/main/rag_prompts.yml
test_prompt initiated
Testing Prompt: default_informative...
rag_get_retriever initiated
rag_get_retriever completed
rag_pipeline initiated
rag_pipeline completed


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

test_prompt completed


Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

evaluate_prompts completed
Dataset({
    features: ['prompt_key', 'prompt', 'question', 'answer', 'ground_truth', 'answer_correctness', 'faithfulness', 'answer_relevancy', 'context_precision', 'context_recall'],
    num_rows: 1
})
The average correctness results have been saved to 'rag_average_correctness.csv'


In [5]:
print("Prompt Text:", best_prompt)


Prompt Text: You are a helpful assistant. Answer any questions solely based on the context provided below. 
If the provided context does not have the relevant facts to answer the question, say "I don't know."

<context>
{context}
</context>



In [6]:
res=gen.setup_retrieval_qa(best_prompt,sample_retriever(),llm,query_rewrite=True)

rag_get_retriever initiated
rag_get_retriever completed
rag_pipeline initiated
rag_pipeline completed


In [7]:
question="who is Marek?"
res.invoke(question)

{'answer': 'Marek is a character who reached the edge of a clearing at night and approached a cabin that holds dark memories for him. His mother disappeared in the forest when he was a boy, and he has a connection to strange occurrences in the woods. He discovers that he and Clara, a young woman he finds in the cabin, are siblings.',
 'context': 'Marek reached the edge of the clearing just as night began to fall. The air felt heavy with an unspoken tension. The cabin was as he remembered it—broken, abandoned, a ghost from his past. He approached cautiously, his breath steady, but his heart racing.\n\nAs he neared the door, it creaked slightly, as if someone had been inside. He gripped the hilt of his dagger, more out of instinct than fear. Stepping into the cabin, he saw the outline of a figure curled by the hearth.\n\nA young woman.\n\n---\nThe cabin held dark memories for Marek. His mother had disappeared in the forest when he was only a boy. His father said she’d gone mad, claiming 

In [13]:
strings = [
    "A young woman.",
    """## Chunk 3: The Locket’s Secret

Clara awoke to a sound—footsteps. Her eyes shot open. In the dim light, she saw a man standing at the door, silhouetted against the night. His green eyes gleamed in the shadows.

“Who are you?” she stammered, clutching her locket.

“I could ask you the same,” the man replied, stepping forward. “This is my family’s cabin. What are you doing here?”
"""
]
documents = dummyRetriever.strings_to_documents(strings)
retriever = dummyRetriever(documents=documents, k=3)

In [14]:
retriever.invoke("Clara")

[Document(page_content='## Chunk 3: The Locket’s Secret\n\nClara awoke to a sound—footsteps. Her eyes shot open. In the dim light, she saw a man standing at the door, silhouetted against the night. His green eyes gleamed in the shadows.\n\n“Who are you?” she stammered, clutching her locket.\n\n“I could ask you the same,” the man replied, stepping forward. “This is my family’s cabin. What are you doing here?”\n')]

In [15]:
pipeline=gen.setup_retrieval_qa(best_prompt,retriever,llm)

rag_pipeline initiated
rag_pipeline completed


In [17]:
pipeline.invoke("Clara",)

{'answer': 'Clara is a character who awoke to the sound of footsteps and found a man with green eyes standing at the door of a cabin. She was startled and clutched her locket when she confronted him.',
 'context': '## Chunk 3: The Locket’s Secret\n\nClara awoke to a sound—footsteps. Her eyes shot open. In the dim light, she saw a man standing at the door, silhouetted against the night. His green eyes gleamed in the shadows.\n\n“Who are you?” she stammered, clutching her locket.\n\n“I could ask you the same,” the man replied, stepping forward. “This is my family’s cabin. What are you doing here?”\n'}